<span style="color:red">
Note: This should not be merged into python-neo, but helps me figure out how to write axonarawio.py. I will add it to the axonarawio branch for now, but should delete it before a PR!
</span>.


# Read axona .bin file

We will go through the steps for reading in Axona dacqUSB system .bin data in continuous format.
The goal is to later implement this in axonarawio.py.

Here is the relevant information from the dacqUSB manual (found here: http://space-memory-navigation.org/DacqUSBFileFormats.pdf):


### Header
The header consists of:
* 4 bytes ID (will be "ADU1", unless the tracker position record is populated with valid data,
in which case it will be "ADU2")
* 4 bytes packet number
* 2 bytes digital inputs
* 2 bytes sync inputs
* 20 bytes tracker position record (only valid data if packet ID is "ADU2") -- same format as
standard .pos file position records.

### Data
Then there are three samples x 64 channels x 16-bits (= 384 bytes), followed by 16
dummy bytes at the end to make up the total packet length of 432. The samples order is
given below. Each sample is two bytes long, in 2's complement.
The data are stored at 48 kHz, so you should have 16000 packets of 432 bytes per second
of recording. Yes, this is very inefficient because you don't have anywhere near 64
channels so it is mostly wasted space; this will be improved in a future version update.
The main complication is that the order of the channels in the data file is not something
nice like 1,2,3, ... Instead, there is a remapping function:
<p>
remap_channels : array [1..64] of word = (
 32, 33, 34, 35, 36, 37, 38, 39,
 0, 1, 2, 3, 4, 5, 6, 7,
 40, 41, 42, 43, 44, 45, 46, 47,
 8, 9, 10, 11, 12, 13, 14, 15,
 48, 49, 50, 51, 52, 53, 54, 55,
 16, 17, 18, 19, 20, 21, 22, 23,
 56, 57, 58, 59, 60, 61, 62, 63,
 24, 25, 26, 27, 28, 29, 30, 31 ); 
</p>

For instance, if you want to find the data for channel 7, you look at remap_channels[7],
which is 38. So, in the 432-byte packet, you ignore the 32 byte header, and the data for
channel 7 will be at:
* bytes 32(header)+(38*2), and 32+(38*2+1) (first sample low and high bytes)
* bytes 32(header)+128(first samples, 64 ch x 2 bytes)+(38*2), and 32+128+(38*2+1) (2nd sample) 
* bytes 32+128+128+(38*2), and 32+128+128+(38*2+1) (third sample) and so on.

### Trailer
Finally, the trailer consists of 16 bytes:
2 bytes contain a record of digital output values
2 bytes contain stimulator status
10 bytes of zeroes (reserved for future use)
2 bytes contain the ASCII keycode if a key was pressed during the time the packet was
active. 

In [1]:
# Imports

import os
import mmap  # python library for memory mapping
import numpy as np  # contains np.memmap for memory mapping (used in python-neo)
import contextlib  # useful for managing contexts

In [2]:
# Set directory and filename

dir_name = r'../../freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw'
base_filename = '20201004_Raw'
suffix = '.bin'

bin_file = os.path.join(dir_name, base_filename + suffix)

print(bin_file)

../../freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/20201004_Raw.bin


In [3]:
# Read the first 432 bytes and look at them

with open(bin_file, 'rb') as f:
    content = f.read(432)

print(content)

b'ADU1\x8d\x04\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe0\xeex\xe88\xeex\xccl\xe2\x84\xea\x1e\x03J\t^\x07l\xe2<\xffb\xfdl\xef\xd6\xee8\xf1\\\xea\x8czJ\xf8\x9a\xfa8\xf1\xc2\x1a\x80\r\xc4\xf0\xd8\x06\x1e\x15\xc0\xf6\xbc\x06\xb0\x03\xce\xfb\x1e\x15\xfa\xef\xf6\rb\x0e\xb2\x06\xf0(|%\n\xed,1\xbcJ\xf0(\xe6\xfe"\x03f\xf9\xd8\x03\x10\x140\x1d\x02\x1a\xb2\x15\xa8A\x10\x14\x86\t\xe0\xf7\x1e\x07\xde\x06x\xf3\x0e\xf4\xa2\xa8\xfc\xca\xce\xedx\xf3j\x11\xa2\x16\x16\x04\x94\xe82\xff\xfc\xe8:\xb1>\x03v\xf82\xff\xea\xdb\xc8\xe7Z\xf0\x9c\xf0\xd8\xfc\x1a\xd7\x0c\xe6\x10\xe9\x1e\xd1\xd8\xfc\xba\xd5\xb6\xe3\x90\xe7P\xe0\x90\xe7j\xe9v\xff\xae\xe9p\xde\x90\xe7l\x12F\x02\x84\x0f\xe4\x11\xce\x06\x1a\xff\xe2\xf0D\xff\xbc\x14\xce\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

Note the substantial amount of wasted space (always assuming 64 channels)

In [4]:
# Read the first 432 * 2 bytes and look at them

with open(bin_file, 'rb') as f:
    content = f.read(432 * 2)

print(content)

b'ADU1\x8d\x04\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe0\xeex\xe88\xeex\xccl\xe2\x84\xea\x1e\x03J\t^\x07l\xe2<\xffb\xfdl\xef\xd6\xee8\xf1\\\xea\x8czJ\xf8\x9a\xfa8\xf1\xc2\x1a\x80\r\xc4\xf0\xd8\x06\x1e\x15\xc0\xf6\xbc\x06\xb0\x03\xce\xfb\x1e\x15\xfa\xef\xf6\rb\x0e\xb2\x06\xf0(|%\n\xed,1\xbcJ\xf0(\xe6\xfe"\x03f\xf9\xd8\x03\x10\x140\x1d\x02\x1a\xb2\x15\xa8A\x10\x14\x86\t\xe0\xf7\x1e\x07\xde\x06x\xf3\x0e\xf4\xa2\xa8\xfc\xca\xce\xedx\xf3j\x11\xa2\x16\x16\x04\x94\xe82\xff\xfc\xe8:\xb1>\x03v\xf82\xff\xea\xdb\xc8\xe7Z\xf0\x9c\xf0\xd8\xfc\x1a\xd7\x0c\xe6\x10\xe9\x1e\xd1\xd8\xfc\xba\xd5\xb6\xe3\x90\xe7P\xe0\x90\xe7j\xe9v\xff\xae\xe9p\xde\x90\xe7l\x12F\x02\x84\x0f\xe4\x11\xce\x06\x1a\xff\xe2\xf0D\xff\xbc\x14\xce\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

At a glance it seems sensible to use f.read() 432 byte snippet by 432 byte snippet since there is no global header or footer. 
Note that the pointer moves with every call to f.read(), if I want to reset it to the beginning I could use f.seek(0)

In [5]:
# How many 432 byte packets does this data contain (<=> num. samples / 3)?

bytes_per_packet = 432

with open(bin_file, 'rb') as f:

    with contextlib.closing(mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)) as mmap_obj:
        num_packets = int(len(mmap_obj)/bytes_per_packet)
        print(num_packets)
        

9600100


In [6]:
# Windows tells me the .bin file is 4147243200  bytes

print('Bytes according to mmap_obj:', num_packets * bytes_per_packet)
print('Bytes according to Windows OS:', 4147243200)

assert(num_packets * bytes_per_packet == 4147243200)

Bytes according to mmap_obj: 4147243200
Bytes according to Windows OS: 4147243200


In [44]:
# Try with np.memmap instead - already gives me the data in ndarray, but still memory mapped

global_header_size = 0
raw_data = np.memmap(bin_file, dtype='int16', mode='r', offset=global_header_size)

print('Size of raw_data:', len(raw_data))
print('Peak at raw_data:', raw_data[16:26])
print('Type of raw_data:', type(raw_data))

Size of raw_data: 2073621600
Peak at raw_data: [ -4384  -6024  -4552 -13192  -7572  -5500    798   2378   1886  -7572]
Type of raw_data: <class 'numpy.memmap'>


In [48]:
# Get data with code snippet from BinConverter, using mmap library:
# https://github.com/HussainiLab/BinConverter/blob/master/BinConverter/core/readBin.py

num_test_samples = num_packets  # read only a few bytes for testing purposes

bytes_per_packet = 432
bytes_of_data = 384
bytes_of_head = 32
bytes_of_tail = 16

with open(bin_file, 'rb') as f:

    with contextlib.closing(mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)) as mmap_obj:
        num_packets = int(len(mmap_obj)/bytes_per_packet)
        
        data = np.ndarray((num_test_samples,), (np.int16, (1,bytes_of_data//2)),  # replace num_test_samples with num_packets
                          mmap_obj[0:bytes_per_packet*num_test_samples],  # ultimately read all
                          bytes_of_head, (bytes_per_packet,))  #.reshape((-1, 1)).flatten()
        #data = samples_to_array(data, channels=channels.tolist())

In [49]:
print(data[0:10])
print(type(data))

[[[-4384 -6024 -4552 ...     0     0     0]]

 [[-3210 -6992  -212 ...     0     0     0]]

 [[ 1076 -7874 -8788 ...     0     0     0]]

 ...

 [[-4784  3388 10956 ...     0     0     0]]

 [[-1540  7270 -4724 ...     0     0     0]]

 [[-8878 -3490 -9256 ...     0     0     0]]]
<class 'numpy.ndarray'>


Is there a clear advantage of one over the other? As it is implemented now, np.memmap seems much faster. And we might actually need the np.memmap object rather than a np.ndarray.

I think I could use np.memmap and then select only the data I want where necessary, e.g. in `_get_analogsignal_chunk`